In [7]:
# Data Visualization

import numpy as np
import open3d as o3d

frame = []



tf_0 = np.load(r'PCD_Data/view00_tf.npy')
tf_obj = np.load(r'PCD_Data/initial_obj_pose.npy')
pcd_0 = o3d.io.read_point_cloud(r"PCD_Data/view00.pcd")


tf_0_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=50.0)
tf_0_frame.transform(tf_0)
object_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0, origin=tf_obj[:3])

frame.append(tf_0_frame)
frame.append(object_frame)

# # Load the numpy file
# tf_0 = np.load(r'PCD_Data/view00_tf.npy')
# tf_1 = np.load(r'PCD_Data/view01_tf.npy')
# tf_2 = np.load(r'PCD_Data/view02_tf.npy')
# tf_3 = np.load(r'PCD_Data/view03_tf.npy')
# tf_4 = np.load(r'PCD_Data/view04_tf.npy')
# tf_5 = np.load(r'PCD_Data/view05_tf.npy')
# tf_6 = np.load(r'PCD_Data/view06_tf.npy')
# tf_7 = np.load(r'PCD_Data/view07_tf.npy')
# tf_8 = np.load(r'PCD_Data/view08_tf.npy')
# tf_obj = np.load(r'PCD_Data/initial_obj_pose.npy')

# # Load pcd files
# pcd_0 = o3d.io.read_point_cloud(r"PCD_Data/view00.pcd")
# # pcd_1 = o3d.io.read_point_cloud(r"PCD_Data/new_tf_down/view01.pcd")
# # pcd_2 = o3d.io.read_point_cloud(r"PCD_Data/new_tf_down/view02.pcd")
# # pcd_3 = o3d.io.read_point_cloud(r"PCD_Data/new_tf_down/view03.pcd")
# # pcd_4 = o3d.io.read_point_cloud(r"PCD_Data/new_tf_down/view04.pcd")


# tf_0_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=50.0)
# tf_0_frame.transform(tf_0)
# tf_1_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0)
# tf_1_frame.transform(tf_1)
# tf_2_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0)
# tf_2_frame.transform(tf_2)
# tf_3_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0)
# tf_3_frame.transform(tf_3)
# tf_4_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0)
# tf_4_frame.transform(tf_4)
# tf_5_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0)
# tf_5_frame.transform(tf_5)
# tf_6_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0)
# tf_6_frame.transform(tf_6)
# tf_7_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0)
# tf_7_frame.transform(tf_7)
# tf_8_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0)
# tf_8_frame.transform(tf_8)
# frame.append(tf_0_frame)
# frame.append(tf_1_frame)
# frame.append(tf_2_frame)
# frame.append(tf_3_frame)
# frame.append(tf_4_frame)
# frame.append(tf_5_frame)
# frame.append(tf_6_frame)
# frame.append(tf_7_frame)
# frame.append(tf_8_frame)


# pasti ada
object_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0, origin=tf_obj[:3])
frame.append(object_frame)

world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0, origin=[0, 0, 0])
frame.append(world_frame)


# test_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0, origin=[517.8511039561226, 163.59750883586588, 600.0])
# frame.append(test_frame)

o3d.visualization.draw_geometries(frame + [pcd_0])
# o3d.visualization.draw_geometries(frame + [pcd_0, pcd_1, pcd_2, pcd_3, pcd_4])



In [11]:
print(tf_obj)

print(tf_0)

[603.03202894 131.01692924   3.28730078   0.          76.25801849
   0.        ]
[[-1.98985365e-04 -9.99999969e-01 -1.47950853e-04  6.02988281e+02]
 [-9.99999888e-01  1.99048766e-04 -4.28638208e-04  1.30926978e+02]
 [ 4.28667644e-04  1.47865543e-04 -9.99999897e-01  4.77014383e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [58]:
# Helper Functions

import open3d as o3d
import numpy as np
import math
import copy
import os


def get_rotation_matrix_z(deg):
    """Creates a 3x3 rotation matrix for the Z-axis."""
    rad = math.radians(deg)
    c, s = math.cos(rad), math.sin(rad)
    return np.array([[c, -s, 0], 
                     [s,  c, 0], 
                     [0,  0, 1]])


def merge_multiview_scan(data_dir, initial_pos, initial_angle, box_size):
    """Merges multiple view PCDs and removes points below the detected plane."""
    pcd_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.pcd') and "view0" in f])
    merged_pcd = o3d.geometry.PointCloud()
    
    obb = o3d.geometry.OrientedBoundingBox(
        center=np.array(initial_pos), 
        R=get_rotation_matrix_z(-initial_angle), 
        extent=np.array(box_size)
    )

    for file_name in pcd_files:
        pcd = o3d.io.read_point_cloud(os.path.join(data_dir, file_name))
   
        # 1. Detect the plane
        plane_model, inliers = pcd.segment_plane(distance_threshold=3.0, ransac_n=3, num_iterations=2000)
        [a, b, c, d] = plane_model

        # 2. Extract all points as a numpy array
        pts = np.asarray(pcd.points)

        # 3. Calculate distance to plane for every point: ax + by + cz + d
        # Points on the plane result in 0. Points above are positive, below are negative.
        distances = a * pts[:, 0] + b * pts[:, 1] + c * pts[:, 2] + d
        
        # 4. Create an index of points that are ABOVE the plane
        # A small offset (e.g., 0.5mm) to avoid keeping table noise
        above_plane_indices = np.where(distances > 0.5)[0]
        pcd = pcd.select_by_index(above_plane_indices)

        # 5. Crop to the OBB and merge
        merged_pcd += pcd.crop(obb)

    return merged_pcd


def preprocess_point_cloud(pcd, num_points):
    """Downsamples, estimates normals, and computes FPFH features."""
    pcd_down = pcd.farthest_point_down_sample(num_points)
    avg_dist = np.mean(pcd_down.compute_nearest_neighbor_distance())
    
    # Estimate Normals
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=avg_dist * 2, max_nn=30))
    
    # Orientation fix: Ensure normals point 'up' (positive Z)
    normals = np.asarray(pcd_down.normals)
    for i in range(len(normals)):
        if normals[i][2] < 0:
            normals[i] *= -1

    # Compute FPFH (Feature descriptors for global matching)
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down, o3d.geometry.KDTreeSearchParamHybrid(radius=avg_dist * 5, max_nn=100))
    
    return pcd_down, fpfh


def run_global_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    """
    Performs RANSAC-based global registration to find a rough alignment.
    """
    distance_threshold = voxel_size * 1.5
    
    # RANSAC based on feature matching
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source, target, source_fpfh, target_fpfh, 
        mutual_filter=True,
        max_correspondence_distance=distance_threshold,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        ransac_n=3, 
        checkers=[
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
        ], 
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
    )
    return result

def run_global_registration_adaptive(source_down, target_down, source_fpfh, target_fpfh):
    """
    Performs iterative RANSAC-based global registration to find the best rough alignment.
    Matches the logic of testing multiple thresholds to find the highest fitness and lowest RMSE.
    """
    max_attempts = 5
    best_fitness = -0.1
    best_inlier_rmse = 100.0
    best_result = None
    best_threshold = None 

    # Thresholds to test, from coarse (10.0) to fine (3.0)
    thresholds = [10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0]
    # thresholds = [7]

    for attempt in range(max_attempts):
        for thr in thresholds:            
            result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
                source_down, target_down, source_fpfh, target_fpfh, 
                mutual_filter=True, # Improved matching
                max_correspondence_distance=thr,
                estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
                ransac_n=3, 
                checkers=[
                    o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.85),
                    o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(thr)
                ], 
                criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(10000, 0.99)
            )

            # Update best result if fitness is good and RMSE is lower
            if result.fitness > 0.85 and result.inlier_rmse < best_inlier_rmse:
                best_fitness = result.fitness
                best_inlier_rmse = result.inlier_rmse
                best_result = result
                best_threshold = thr
            
            # Early exit if we find a very high-quality match
            if best_fitness > 0.95 and best_inlier_rmse < 2.35: 
                print(f"Excellent Global Fit Found at Threshold {best_threshold}")
                return best_result, best_threshold
            
    if best_result is None:
        print("Warning: RANSAC could not find a fit above 0.85 fitness.")
        # Fallback to the last result generated if nothing met the 0.85 criteria
        return result, thr

    print(f"RANSAC Finished. Best Threshold: {best_threshold} | Fitness: {best_fitness:.4f}")
    return best_result, best_threshold


# One time local refinement using ICP
def run_local_refinement(source, target, initial_transformation, voxel_size):
    """
    Performs ICP registration to refine the alignment found by RANSAC.
    """
    # We use a smaller threshold for ICP to ensure high precision
    distance_threshold = voxel_size * 0.4
    
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, initial_transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000)
    )
    return result


# Progressive Local Refinement using ICP
def run_local_refinement_adaptive(source, target, initial_trans, best_ransac_thr):
    """
    Refines alignment using an iterative ICP loop. 
    Starts with a coarse threshold and progressively tightens for precision.
    """
    # Define a range of multipliers to tighten the search radius
    # multipliers = [1.0, 0.8, 0.5, 0.25]
    multipliers = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
    # multipliers = [0.6]
    thresholds = [best_ransac_thr * m for m in multipliers]
    
    best_result = None
    best_inlier_rmse = float('inf') # Start with the highest possible error
    
    print(f"{'Threshold':<12} | {'Fitness':<12} | {'RMSE':<12}")
    print("-" * 45)

    for thr in thresholds:
        # Execute Point-to-Point ICP
        reg_icp = o3d.pipelines.registration.registration_icp(
            source, target, thr, initial_trans,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000)
        )

        # Logging the current iteration results
        print(f"{thr:<12.2f} | {reg_icp.fitness:<12.4f} | {reg_icp.inlier_rmse:<12.4f}")

        # Selection Logic: Prioritize lowest RMSE (highest precision) 
        # as long as the fitness is acceptable (e.g., > 85%)
        if reg_icp.fitness > 0.85 and reg_icp.inlier_rmse < best_inlier_rmse:
            best_inlier_rmse = reg_icp.inlier_rmse
            best_result = reg_icp

    # Fallback: if no run met the 85% fitness, return the last result
    return best_result if best_result is not None else reg_icp

In [59]:
# Main Processing Pipeline

# --- 1. Configuration ---
DATA_DIR = "PCD_Data"
SOURCE_PATH = "workpiece/z_bracket_10000.pcd" #  CAD model

tf_obj = np.load(r'PCD_Data/initial_obj_pose.npy')
YOLO_POS = tf_obj[:3]                                   # Initial guess from YOLO
YOLO_ANGLE = tf_obj[4]                                  # Initial angle from YOLO
CROP_BOX = [100, 100, 80]                               # Region of interest size


# --- 2. Data Preparation ---
print("Step 1: Merging and cleaning scans...")
source_cloud = o3d.io.read_point_cloud(SOURCE_PATH)
target_cloud = merge_multiview_scan(DATA_DIR, YOLO_POS, YOLO_ANGLE, CROP_BOX)
o3d.visualization.draw_geometries([target_cloud], window_name="Merged Target Cloud")

# Get downsampled versions and features for RANSAC
source_down, source_fpfh = preprocess_point_cloud(source_cloud, 10000)
target_down, target_fpfh = preprocess_point_cloud(target_cloud, 10000)
o3d.visualization.draw_geometries([target_down], window_name="Downsampled Target Cloud")


# --- 3. Global Alignment (RANSAC) ---
print("Step 2: Running RANSAC Global Registration...")
ransac_res, best_thr = run_global_registration_adaptive(source_down, target_down, source_fpfh, target_fpfh)
print(ransac_res)

# Visualize RANSAC result
source_temp = copy.deepcopy(source_cloud)
source_temp.transform(ransac_res.transformation)
source_temp.paint_uniform_color([1, 0, 0])
target_down.paint_uniform_color([0, 0.651, 0.929])
o3d.visualization.draw_geometries([source_temp, target_down], window_name="RANSAC Result")


Step 1: Merging and cleaning scans...
Step 2: Running RANSAC Global Registration...
[Open3D WARNING] Too few correspondences (183) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (183) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (183) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (183) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (183) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (183) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (183) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (183) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (183) after mutual 

In [60]:
# --- 4. Local Alignment (ICP) ---
print("Step 3: Running ICP Local Refinement...")
icp_res = run_local_refinement_adaptive(source_cloud, target_down, ransac_res.transformation, best_thr)
print(icp_res)

# --- 5. Extract Final Results ---
final_matrix = icp_res.transformation
initial_POS = final_matrix[:3, 3] 
initial_ORI = final_matrix[:3, :3] 
print("="*30)
print(f"Final Position: {initial_POS}")
print(f"Final Orientation Matrix:\n{initial_ORI}")
print(f"Fitness: {icp_res.fitness:.4f}")
print(f"RMSE: {icp_res.inlier_rmse:.4f}")


# --- 6. Final Visualization ---
source_final = copy.deepcopy(source_cloud).transform(final_matrix)
source_final.paint_uniform_color([1, 0, 0])         # Red: CAD Model
target_cloud.paint_uniform_color([0, 0.65, 0.93])   # Blue: Scanned Data
o3d.visualization.draw_geometries([source_final, target_cloud])

Step 3: Running ICP Local Refinement...
Threshold    | Fitness      | RMSE        
---------------------------------------------
6.00         | 0.8637       | 2.2664      
5.40         | 0.8406       | 2.0913      
4.80         | 0.8154       | 1.9262      
4.20         | 0.7876       | 1.7657      
3.60         | 0.7578       | 1.6254      
3.00         | 0.7177       | 1.4761      
2.40         | 0.6683       | 1.3400      
1.80         | 0.5831       | 1.2169      
1.20         | 0.2888       | 0.9150      
0.60         | 0.0407       | 0.4501      
RegistrationResult with fitness=8.637000e-01, inlier_rmse=2.266406e+00, and correspondence_set size of 8637
Access transformation to get result.
Final Position: [607.60751859 -15.10481988  -2.96682283]
Final Orientation Matrix:
[[ 0.39911199  0.91666784 -0.02072885]
 [-0.91637447  0.39801344 -0.04293171]
 [-0.03110375  0.03612994  0.99886295]]
Fitness: 0.8637
RMSE: 2.2664
